In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score,StratifiedKFold,RepeatedStratifiedKFold, cross_validate
from copy import deepcopy

from time import time
from sklearn.preprocessing import FunctionTransformer

In [33]:
def train_model(list_model,X_train,y_train,X_test,y_test,metric,cv,scorer,pipeline):
    df_model = pd.DataFrame(columns = ["model_name","set_data","score","model"])
    set_data = ["test","cv","train"]

    for m in list_model: 
        pipeline_copy = deepcopy(pipeline)
        pipeline_copy.set_params(model = list_model[m])
        spot_check = cross_val_score(pipeline_copy,X_train,y_train,cv = cv,scoring = scorer,n_jobs= -1)
        spot_check = spot_check.mean()

        model = pipeline_copy.fit(X_train,y_train)
        score = metric(y_test,model.predict(X_test),squared = False)
        score_train = metric(y_train,model.predict(X_train),squared = False)
        
        model_list = [m] * 3
        tes = pd.DataFrame(list(zip(model_list,set_data,[score,spot_check,score_train],[model,model,model])),columns = ["model_name","set_data","score","model"])
        print(f"model {m} selesai di training")
        print(f"score test {score}")
        print(f"score cv {spot_check}")
        print(f"score train {score_train}")
        print("=====================================")
        df_model = pd.concat([df_model,tes],ignore_index = True)
        
    return df_model

def rfecv(X, y, pipeline,min_features_to_select=3, cv = 3,step=3,scoring_metric="f1",scoring_decimals=3,random_state=42,groups = None):
    # Initialize survivors and ranked list
    estimator = deepcopy(pipeline)
    estimator.steps.pop(-1)
    survivors = estimator.fit_transform(X,y).columns.tolist()
    ranks = []
    scores = []
    while len(survivors) >= min_features_to_select:
        remove_column_transformer = FunctionTransformer(lambda x: x.drop(ranks, axis=1))
        estimator = deepcopy(pipeline)
        estimator.steps.insert(-1, ('remove_column_transformer', remove_column_transformer))
        print("[%.2f] evaluating %i features ..." % (time(), len(survivors)))
        cv_result = cross_validate(estimator, X, y,
                                cv=cv,
                                groups = groups,
                                scoring=scoring_metric,
                                return_estimator=True)
        score = np.mean(cv_result["test_score"])
        if scoring_decimals is None:
            scores.append(score)
        else:
            scores.append(round(score, scoring_decimals))            
        print("[%.2f] ... score %f." % (time(), scores[-1]))
        
        best_estimator = cv_result["estimator"][np.argmax(cv_result["test_score"])]
        if isinstance(best_estimator, Pipeline):
            weights = best_estimator[-1].feature_importances_
        else:
            weights = best_estimator.feature_importances_
        weights = list(np.power(weights, 2))
        for _ in range(max(min(step, len(survivors) - min_features_to_select), 1)):
            idx = np.argmin(weights)
            ranks.insert(0, survivors.pop(idx))
            weights.pop(idx)
    ranks_reverse = list(reversed(ranks))
    last_max_idx = len(scores) - np.argmax(list(reversed(scores))) - 1
    removed_features = set(ranks_reverse[0:last_max_idx * step])
    best_features = [f for f in X.columns if f not in removed_features]
    return best_features, max(scores), ranks, scores

In [10]:
df = pd.read_csv('../../data/processed/data_clean.csv',index_col=0)
df.head()

,Make,Vehicle_Class,Engine_Size,Cylinders,Transmission,Fuel_Type,Fuel_Consumption_City,Fuel_Consumption_Hwy,Fuel_Consumption_Comb,CO2_Emissions
Id,,,,,,,,,,
1,FOLD,PICKUP TRUCK - STANDARD,3.5,6.0,A6,X,8.64,6.14,7.514791,306.0
2,CHEVO,PICKUP TRUCK - STANDARD,5.3,NaN,A6,E,27.27,30.76,28.840000,283.0
3,BMV,SUBCOMPACT,4.4,NaN,M6,Z,17.00,2.03,NaN,329.0
4,KIO,SUV - SMALL,NaN,4.0,AS6,X,6.52,15.90,NaN,270.0
5,BARUSU,MINICOMPACT,3.0,6.0,NaN,Z,20.15,6.00,13.800000,193.0


In [14]:
## stratified shuffle
X = df.drop(columns="CO2_Emissions")

y = df['CO2_Emissions']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((109712, 9), (27429, 9), (109712,), (27429,))

In [15]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer,TargetEncoder

cat_columns = X_train.select_dtypes(include=['object']).columns.tolist()
num_columns = X_train.select_dtypes(include=['int64','float64']).columns.tolist()
num_columns = num_columns

num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", PowerTransformer())
])

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", TargetEncoder(target_type="continuous"))
])

preprocessor = ColumnTransformer([
("numerical",num_pipe,num_columns),
("categorical",cat_pipe,cat_columns)
],remainder = "drop",verbose_feature_names_out=False).set_output(transform="pandas")

In [18]:
prep_pipeline = Pipeline([
    ("preprocessor",preprocessor)
])
prep_pipeline.fit_transform(X_train,y_train).head()

,Engine_Size,Cylinders,Fuel_Consumption_City,Fuel_Consumption_Hwy,Fuel_Consumption_Comb,Make,Vehicle_Class,Transmission,Fuel_Type
Id,,,,,,,,,
13536,1.919337,1.607482,2.378615,1.181489,2.024805,233.252510,317.790117,284.398806,237.638872
113916,0.716210,0.347239,-0.408942,1.561046,0.616546,274.424523,212.907008,234.916039,237.957920
15762,-0.419030,-0.909586,-0.565152,1.276891,0.690425,273.312011,318.180132,280.536918,237.411056
39814,-0.938835,-0.909586,1.978605,0.533004,0.138299,266.187663,212.856233,261.050545,275.998263
67554,-0.302979,-0.909586,-0.076850,0.421846,0.538363,221.097393,243.115913,249.644551,237.638872


In [38]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor, VotingRegressor

from sklearn.model_selection import KFold


pipeline = Pipeline([
    ("preprocess",preprocessor),
    ("model",None)
])

list_model = {
    "lgbm":LGBMRegressor(verbose=-1,random_state=42),
    "xgb":XGBRegressor(verbosity=0,random_state=42,tree_method="gpu_hist",gpu_id=1),
    'catboost' : CatBoostRegressor(verbose=0,random_state=42,n_estimators=200),
    'stc' : StackingRegressor(estimators=[('lgbm', LGBMRegressor(verbose=-1,random_state=42)),
                                           ('xgb', XGBRegressor(verbosity=0,random_state=42,tree_method="gpu_hist",gpu_id=1)),
                                           ('catboost', CatBoostRegressor(verbose=0,random_state=42,n_estimators=200))],
                               final_estimator=LGBMRegressor(verbose=-1,random_state=42)),
    'voting' : VotingRegressor(estimators=[('lgbm', LGBMRegressor(verbose=-1,random_state=42)),
                                           ('xgb', XGBRegressor(verbosity=0,random_state=42,tree_method="gpu_hist",gpu_id=1)),
                                           ('catboost', CatBoostRegressor(verbose=0,random_state=42,n_estimators=200))])
}
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer


cv = KFold(n_splits=3,shuffle=True,random_state=42)
df_model = train_model(list_model,X_train,y_train,X_test,y_test,mean_squared_error,cv,'neg_root_mean_squared_error',pipeline)

model lgbm selesai di training
score test 50.75968245831882
score cv -49.2525608596099
score train 48.54446587503131


C:\Users\VICTUS\AppData\Local\Temp\ipykernel_31448\4232660393.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_model = pd.concat([df_model,tes],ignore_index = True)


model xgb selesai di training
score test 50.51267264173339
score cv -49.64304173850626
score train 46.37867767456576
model catboost selesai di training
score test 50.39153166153459
score cv -49.05041922042482
score train 47.67845762202924
model stc selesai di training
score test 50.15446608720437
score cv -48.83355627629292
score train 47.4651685566859
model voting selesai di training
score test 50.18997237546574
score cv -48.85560244975798
score train 47.27058562179182
